In [1]:
import cv2
import os

def draw_boxes_on_image(image_path, label_path, output_path):
    
    image = cv2.imread(image_path)
    
    with open(label_path, 'r') as f:
        labels = f.readlines()[2:] 

    for label in labels:
        parts = label.strip().split()
        x1, y1, x2, y2, x3, y3, x4, y4 = map(float, parts[:8])  
        
        points = [(int(x1), int(y1)), (int(x2), int(y2)), (int(x3), int(y3)), (int(x4), int(y4))]
        
        for i in range(4):
            cv2.line(image, points[i], points[(i + 1) % 4], (0, 255, 0), 2)
        
       
        cv2.circle(image, (int(x1), int(y1)), 5, (0, 255, 255), -1) 
    
    cv2.imwrite(output_path, image)

def process_dataset(images_folder, labels_folder, output_folder):
 
    os.makedirs(output_folder, exist_ok=True)
    
    for image_filename in os.listdir(images_folder):
        if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
            
            label_filename = os.path.splitext(image_filename)[0] + '.txt'
            
            
            image_path = os.path.join(images_folder, image_filename)
            label_path = os.path.join(labels_folder, label_filename)
            output_path = os.path.join(output_folder, image_filename)
            
            if os.path.exists(label_path):
                draw_boxes_on_image(image_path, label_path, output_path)
            else:
                print(f"Label file not found for {image_filename}")

In [ ]:
# Example usage
images_folder = 'training_images'
labels_folder = "
output_folder = 'Output_images_without_names'

process_dataset(images_folder, labels_folder, output_folder)

In [ ]:
import cv2
import os

def reduce_resolution_and_blur_for_folder(images_folder, output_folder, scale_percent=30, blur_ksize=(7, 7)):
    
    os.makedirs(output_folder, exist_ok=True)

    for image_filename in os.listdir(images_folder):
        if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
            
            image_path = os.path.join(images_folder, image_filename)
            output_path = os.path.join(output_folder, image_filename)
            
            image = cv2.imread(image_path)
            
            original_height, original_width = image.shape[:2]
            
            new_width = int(original_width * scale_percent / 100)
            new_height = int(original_height * scale_percent / 100)
            
            resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
            
            blurred_image = cv2.GaussianBlur(resized_image, blur_ksize, 0)
            
            cv2.imwrite(output_path, blurred_image)

            print(f"Processed {image_filename}")

images_folder = 'part1/images'
output_folder = 'reduced_images'

reduce_resolution_and_blur_for_folder(images_folder, output_folder)

Processed P0003.png
Processed P0004.png
Processed P0007.png
Processed P0019.png
Processed P0027.png
Processed P0047.png
Processed P0053.png
Processed P0056.png
Processed P0060.png
Processed P0075.png
Processed P0079.png
Processed P0081.png
Processed P0086.png
Processed P0104.png
Processed P0110.png
Processed P0117.png
Processed P0123.png
Processed P0128.png
Processed P0130.png
Processed P0131.png
Processed P0143.png
Processed P0146.png
Processed P0150.png
Processed P0161.png
Processed P0168.png
Processed P0170.png
Processed P0173.png
Processed P0175.png
Processed P0179.png
Processed P0186.png
Processed P0189.png
Processed P0194.png
Processed P0198.png
Processed P0199.png
Processed P0206.png
Processed P0213.png
Processed P0217.png
Processed P0228.png
Processed P0237.png
Processed P0242.png
Processed P0246.png
Processed P0249.png
Processed P0257.png
Processed P0259.png
Processed P0261.png
Processed P0262.png
Processed P0300.png
Processed P0305.png
Processed P0309.png
Processed P0329.png


# This is used to convert OBB format to YOLO object detection format

In [ ]:
import os
import numpy as np
from PIL import Image

# def convert_to_yolo_format(x1, y1, x2, y2, x3, y3, x4, y4):
#     # Calculate the center of the bounding box
#     x_center = (x1 + x2 + x3 + x4) / 4.0
#     y_center = (y1 + y2 + y3 + y4) / 4.0
    
#     # Calculate width and height
#     width = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
#     height = np.sqrt((x3 - x2)**2 + (y3 - y2)**2)
    
#     return x_center, y_center, width, height
def convert_to_yolo_format(x1, y1, x2, y2, x3, y3, x4, y4):
    xmin = min(x1, x2, x3, x4)
    ymin = min(y1, y2, y3, y4)
    xmax = max(x1, x2, x3, x4)
    ymax = max(y1, y2, y3, y4)
    
    
    x_center = (xmin + xmax) / 2.0
    y_center = (ymin + ymax) / 2.0
    
    
    width = xmax - xmin
    height = ymax - ymin
    
    return x_center, y_center, width, height

category_mapping = {
    "plane": 0,
    "ship": 1,
    "storage-tank": 2,
    "baseball-diamond": 3,
    "tennis-court": 4,
    "basketball-court": 5,
    "ground-track-field": 6,
    "harbor": 7,
    "bridge": 8,
    "large-vehicle": 9,
    "small-vehicle": 10,
    "helicopter": 11,
    "roundabout": 12,
    "soccer-ball-field": 13,
    "swimming-pool": 14
}

def process_labels(labels_folder, images_folder):
    output_folder = os.path.join("training_yolo_labels")
    os.makedirs(output_folder, exist_ok=True)
    
    
    for file_name in os.listdir(labels_folder):
        if file_name.endswith(".txt"):
            
            image_file = file_name.replace(".txt", ".png")  
            image_path = os.path.join(images_folder, image_file)
            
            if not os.path.exists(image_path):
                print(f"Warning: Image {image_file} not found, skipping label file {file_name}")
                continue
            
            
            with Image.open(image_path) as img:
                img_width, img_height = img.size  
            
            with open(os.path.join(labels_folder, file_name), "r") as f:
                lines = f.readlines()[2:] 
            
            yolo_data = []
            for line in lines:
                data = line.strip().split()
                
                x1, y1, x2, y2, x3, y3, x4, y4 = map(float, data[:8])
                category_name = data[8]
                
                class_id = category_mapping.get(category_name, -1)
                if class_id == -1:
                    continue  
                
                x_center, y_center, width, height = convert_to_yolo_format(x1, y1, x2, y2, x3, y3, x4, y4)
                
                x_center /= img_width
                y_center /= img_height
                width /= img_width
                height /= img_height
                
                yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")
            
            output_file = os.path.join(output_folder, file_name)
            with open(output_file, "w") as out_f:
                out_f.write("\n".join(yolo_data))

# Run the function
process_labels("training_labels", "training_images")


# This one is to exclude some classes that are big or low in occurence

In [1]:
import os
import numpy as np
from PIL import Image

# Convert OBB (Oriented Bounding Box) to a simple bounding box format
def convert_to_yolo_format(x1, y1, x2, y2, x3, y3, x4, y4):
    xmin = min(x1, x2, x3, x4)
    ymin = min(y1, y2, y3, y4)
    xmax = max(x1, x2, x3, x4)
    ymax = max(y1, y2, y3, y4)
    
    x_center = (xmin + xmax) / 2.0
    y_center = (ymin + ymax) / 2.0
    
    width = xmax - xmin
    height = ymax - ymin
    
    return x_center, y_center, width, height

# Updated class mapping: Only the selected classes are included
category_mapping = {
    "plane": 0,
    "ship": 1,
    "harbor": 2,
    "bridge": 3,
    "large-vehicle": 4,
    "small-vehicle": 5,
    "helicopter": 6,
    "swimming-pool": 7
}

# Process the labels and images to convert them to YOLO format
def process_labels(labels_folder, images_folder):
    output_folder = os.path.join("train_labels_in_yolo_format")
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(labels_folder):
        if file_name.endswith(".txt"):
            
            # Get the corresponding image file name
            image_file = file_name.replace(".txt", ".png")
            image_path = os.path.join(images_folder, image_file)
            
            # If the image doesn't exist, skip this label file
            if not os.path.exists(image_path):
                print(f"Warning: Image {image_file} not found, skipping label file {file_name}")
                continue
            
            with Image.open(image_path) as img:
                img_width, img_height = img.size
            
            with open(os.path.join(labels_folder, file_name), "r") as f:
                lines = f.readlines()[2:]  # Skip the first 2 lines
            
            yolo_data = []
            for line in lines:
                data = line.strip().split()
                
                # Get the coordinates of the bounding box and the category name
                x1, y1, x2, y2, x3, y3, x4, y4 = map(float, data[:8])
                category_name = data[8]
                
                # Check if the class is in the selected mapping
                class_id = category_mapping.get(category_name, -1)
                if class_id == -1:
                    continue  # Skip this label if it's not in the selected categories
                
                # Convert to YOLO format (normalized coordinates)
                x_center, y_center, width, height = convert_to_yolo_format(x1, y1, x2, y2, x3, y3, x4, y4)
                
                # Normalize the values
                x_center /= img_width
                y_center /= img_height
                width /= img_width
                height /= img_height
                
                yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")
            
            # Write the YOLO labels to the output folder
            output_file = os.path.join(output_folder, file_name)
            with open(output_file, "w") as out_f:
                out_f.write("\n".join(yolo_data))

# Run the function to process labels
process_labels("training_images", "training_labels")


# Below is to show images on dataset

In [ ]:
import os
import cv2
import numpy as np

image_folder = 'training_images'
label_folder = 'training_labels_in_yolo_format'
output_folder = 'object_detection_label_images'

os.makedirs(output_folder, exist_ok=True)

def draw_yolo_labels(image, label_file):
    with open(label_file, 'r') as f:
        labels = f.readlines()

    for label in labels:
        
        parts = label.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        
        img_height, img_width = image.shape[:2]

        
        x_center_pixel = int(x_center * img_width)
        y_center_pixel = int(y_center * img_height)
        width_pixel = int(width * img_width)
        height_pixel = int(height * img_height)

        
        top_left_x = int(x_center_pixel - width_pixel / 2)
        top_left_y = int(y_center_pixel - height_pixel / 2)
        bottom_right_x = int(x_center_pixel + width_pixel / 2)
        bottom_right_y = int(y_center_pixel + height_pixel / 2)

        
        color = (0, 255, 0) 
        thickness = 2  
        cv2.rectangle(image, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)

    return image

for image_name in os.listdir(image_folder):
    
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, image_name)
        label_path = os.path.join(label_folder, image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))

        if os.path.exists(label_path):
            
            image = cv2.imread(image_path)

            annotated_image = draw_yolo_labels(image, label_path)

            output_image_path = os.path.join(output_folder, image_name)
            cv2.imwrite(output_image_path, annotated_image)
            print(f"Processed and saved: {output_image_path}")
        else:
            print(f"Label file missing for image: {image_name}")


Processed and saved: object_detection_label_images\P0000.png
Processed and saved: object_detection_label_images\P0001.png
Processed and saved: object_detection_label_images\P0002.png
Processed and saved: object_detection_label_images\P0005.png
Processed and saved: object_detection_label_images\P0008.png
Processed and saved: object_detection_label_images\P0010.png
Processed and saved: object_detection_label_images\P0011.png
Processed and saved: object_detection_label_images\P0012.png
Processed and saved: object_detection_label_images\P0013.png
Processed and saved: object_detection_label_images\P0018.png
Processed and saved: object_detection_label_images\P0020.png
Processed and saved: object_detection_label_images\P0021.png
Processed and saved: object_detection_label_images\P0022.png
Processed and saved: object_detection_label_images\P0023.png
Processed and saved: object_detection_label_images\P0025.png
Processed and saved: object_detection_label_images\P0029.png
Processed and saved: obj

### Draw square on images


In [1]:
from PIL import Image, ImageDraw

# Open the large image
large_image_path = "Output_images_without_names/P0030.png"  # Replace with your image file path
large_image = Image.open(large_image_path)

# Define the size of the square
square_size = 1024

# Get the dimensions of the large image
width, height = large_image.size

# Calculate how many squares can fit horizontally and vertically
num_squares_x = width // square_size
num_squares_y = height // square_size

# Create a drawing context to draw on the image
draw = ImageDraw.Draw(large_image)

# Draw squares side by side
for i in range(num_squares_x):
    for j in range(num_squares_y):
        # Calculate the top-left corner of the square
        top_left_x = i * square_size
        top_left_y = j * square_size
        bottom_right_x = top_left_x + square_size
        bottom_right_y = top_left_y + square_size
        
        # Draw the rectangle (square) on the image
        draw.rectangle([top_left_x, top_left_y, bottom_right_x, bottom_right_y], outline="red", width=5)  # You can adjust the outline color/width

# Save or display the image with drawn squares
large_image.save("output_image_with_squares.jpg")
